In [14]:
import yfinance as yf
import pandas as pd
import numpy as np

novo = yf.Ticker("NVO")

novo.history(period = "3y")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-04-12 00:00:00-04:00,33.371405,33.486663,33.275360,33.356998,1147400,0.0,0.0
2021-04-13 00:00:00-04:00,33.866045,34.048536,33.760397,33.986103,1627200,0.0,0.0
2021-04-14 00:00:00-04:00,34.202207,34.221416,33.731576,33.880451,1913000,0.0,0.0
2021-04-15 00:00:00-04:00,34.182995,34.523960,34.182995,34.341473,1944200,0.0,0.0
2021-04-16 00:00:00-04:00,34.668031,34.773683,34.533566,34.658424,2641200,0.0,0.0
...,...,...,...,...,...,...,...
2024-04-03 00:00:00-04:00,125.379997,128.130005,125.379997,126.989998,3616400,0.0,0.0
2024-04-04 00:00:00-04:00,126.900002,127.480003,123.769997,123.959999,4006800,0.0,0.0
2024-04-05 00:00:00-04:00,124.669998,126.169998,124.599998,125.860001,4995400,0.0,0.0


In [18]:
data = yf.download(["^GSPC", "NVO", "MSFT"], start="2021-04-08", end="2024-04-08", interval="1d")

# Selecting only the "Adj Close" prices for all symbols
data = data["Adj Close"]

# Calculate percentage change for the S&P 500 index
data['Sp500_returns'] = data['^GSPC'].pct_change() * 100

data["msft_returns"] = data["MSFT"].pct_change() * 100
data["nvo_returns"] = data["NVO"].pct_change() * 100

# Drop NaN values

data = data.dropna()


varianceMicrosoft = data["msft_returns"].var()
sdMicrosoft = np.sqrt(varianceMicrosoft)

varianceNovo = data["nvo_returns"].var()
sdNovo = np.sqrt(varianceNovo)

varianceSP500 = data["Sp500_returns"].var()
sdSp500 = np.sqrt(varianceSP500)

# Display results
print("Microsoft - Variance:", varianceMicrosoft, " Standard Deviation:", sdMicrosoft)
print("Novo - Variance:", varianceNovo, " Standard Deviation:", sdNovo)
print("S&P 500 - Variance:", varianceSP500, " Standard Deviation:", sdSp500)

[*********************100%***********************]  3 of 3 completed
Microsoft - Variance: 2.9779675682024034  Standard Deviation: 1.7256788716914868
Novo - Variance: 3.539586886118516  Standard Deviation: 1.8813789852442053
S&P 500 - Variance: 1.188950547110858  Standard Deviation: 1.0903900894225231


In [25]:
covariance_msft_sp500 = data['msft_returns'].cov(data['Sp500_returns'])
beta_msft = covariance_msft_sp500 / varianceSP500

covariance_nvo_sp500 = data['nvo_returns'].cov(data['Sp500_returns'])
beta_nvo = covariance_nvo_sp500 / varianceSP500

print("Beta for Microsoft (MSFT):", beta_msft)
print("Beta for Novo Nordisk (NVO):", beta_nvo)

Beta for Microsoft (MSFT): 1.2537342903241118
Beta for Novo Nordisk (NVO): 0.5336573638021458


In [24]:
import statsmodels.api as sm


X = sm.add_constant(data['Sp500_returns'])


y_msft = data['msft_returns']
y_nvo = data['nvo_returns']

model_msft = sm.OLS(y_msft, X, missing='drop')
results_msft = model_msft.fit()
beta_msft_ols = results_msft.params[1]


model_nvo = sm.OLS(y_nvo, X, missing='drop')
results_nvo = model_nvo.fit()
beta_nvo_ols = results_nvo.params[1]

print("Beta for Microsoft (MSFT) using OLS regression:", beta_msft_ols)
print("Beta for Novo Nordisk (NVO) using OLS regression:", beta_nvo_ols)

Beta for Microsoft (MSFT) using OLS regression: 1.2537342903241122
Beta for Novo Nordisk (NVO) using OLS regression: 0.5336573638021459
